**Working Model**

In [19]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import networkx as nx
import random
import matplotlib.pyplot as plt

# Define top 15 towns in Kenya
towns = ['Nairobi', 'Mombasa', 'Nakuru', 'Eldoret', 'Kisumu', 'Nyeri', 'Machakos', 'Kericho', 'Meru', 'Kitale', 'Busia', 'Garissa', 'Isiolo', 'Narok', 'Malindi']

# Generate routes ensuring each town is connected to at least three others
def generate_routes(towns):
    routes = []
    for town in towns:
        destinations = random.sample([t for t in towns if t != town], min(3, len(towns)-1))
        for dest in destinations:
            routes.append((town, dest))
            routes.append((dest, town))  # Bidirectional route
    return routes

routes = generate_routes(towns)

# Define start and end date for 2 years of data
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 1, 1)

# Generate dates within the 2-year period
dates = pd.date_range(start_date, end_date, freq='D')

# Function to generate random crime data for each route
def generate_crime_data():
    data = []
    route_id = 1

    for route in routes:
        origin, destination = route
        for date in dates:
            crime_occurred = np.random.choice([0, 1], p=[0.8, 0.2])  # 20% chance a crime occurred
            crime_severity = np.random.choice([1, 2, 3], p=[0.6, 0.3, 0.1]) if crime_occurred else 0
            day_of_week = date.strftime('%A')

            data.append({
                'Route_ID': route_id,
                'Route_Name': f"{origin} - {destination}",
                'Origin': origin,
                'Destination': destination,
                'Date': date,
                'Day_of_Week': day_of_week,
                'Crime_Occurrence': crime_occurred,
                'Crime_Severity': crime_severity
            })

        route_id += 1

    return pd.DataFrame(data)

# Generate the data
crime_data = generate_crime_data()

# Convert 'Day_of_Week' to numerical values (e.g., Monday = 0, Tuesday = 1)
day_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
crime_data['Day_of_Week'] = crime_data['Day_of_Week'].map(day_map)

# Prepare features and target variable
X = crime_data[['Route_ID', 'Day_of_Week']]
y = crime_data['Crime_Occurrence']

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Check accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

def build_graph(crime_data):
    G = nx.Graph()

    # Add edges with weights (average crime severity as weight)
    for _, row in crime_data.iterrows():
        if row['Crime_Occurrence'] == 1:
            G.add_edge(row['Origin'], row['Destination'], weight=row['Crime_Severity'])

    return G

def update_graph_weights(G, clf, day_of_week):
    day_num = day_map[day_of_week]

    for edge in G.edges:
        route_id = crime_data[(crime_data['Origin'] == edge[0]) & (crime_data['Destination'] == edge[1])]['Route_ID'].unique()
        if route_id.size > 0:  # Check if route_id has any elements
            X_new = np.array([[route_id[0], day_num]])
            crime_pred = clf.predict(X_new)[0]
            if crime_pred == 0:
                G[edge[0]][edge[1]]['weight'] = 0
            else:
                G[edge[0]][edge[1]]['weight'] = crime_data[(crime_data['Origin'] == edge[0]) &
                                                             (crime_data['Destination'] == edge[1])]['Crime_Severity'].mean()

def find_safest_route(G, origin, destination):
    try:
        path = nx.dijkstra_path(G, source=origin, target=destination, weight='weight')
        return path
    except nx.NetworkXNoPath:
        return "No available route between these towns."

# Build and update graph
G = build_graph(crime_data)

# Example: Update graph and find the safest route from Nairobi to Mombasa on a Tuesday
update_graph_weights(G, clf, 'Tuesday')
safest_route = find_safest_route(G, 'Nairobi', 'Mombasa')
print(f"The safest route from Nairobi to Mombasa on Tuesday is: {safest_route}")

# Visualization

def draw_graph(G, path=None):
    pos = nx.spring_layout(G)  # Position nodes using the Fruchterman-Reingold force-directed algorithm

    plt.figure(figsize=(12, 8))

    # Draw nodes and edges
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

    if path:
        # Highlight the safest path
        path_edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='r', width=2.0)

    nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif')

    plt.title('Graph Visualization with Safest Route')
    plt.show()

# Draw the graph and highlight the safest route
draw_graph(G, safest_route)


In [20]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import networkx as nx
import random
import matplotlib.pyplot as plt

# Define top 15 towns in Kenya
towns = ['Nairobi', 'Mombasa', 'Nakuru', 'Eldoret', 'Kisumu', 'Nyeri', 'Machakos', 'Kericho', 'Meru', 'Kitale', 'Busia', 'Garissa', 'Isiolo', 'Narok', 'Malindi']

# Generate routes ensuring each town is connected to at least three others
def generate_routes(towns):
    routes = []
    for town in towns:
        destinations = random.sample([t for t in towns if t != town], min(3, len(towns)-1))
        for dest in destinations:
            routes.append((town, dest))
            routes.append((dest, town))  # Bidirectional route
    return routes

routes = generate_routes(towns)

# Define start and end date for 2 years of data
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 1, 1)

# Generate dates within the 2-year period
dates = pd.date_range(start_date, end_date, freq='D')

# Function to generate random crime data for each route
def generate_crime_data():
    data = []
    route_id = 1

    for route in routes:
        origin, destination = route
        for date in dates:
            crime_occurred = np.random.choice([0, 1], p=[0.8, 0.2])  # 20% chance a crime occurred
            crime_severity = np.random.choice([1, 2, 3], p=[0.6, 0.3, 0.1]) if crime_occurred else 0
            day_of_week = date.strftime('%A')

            data.append({
                'Route_ID': route_id,
                'Route_Name': f"{origin} - {destination}",
                'Origin': origin,
                'Destination': destination,
                'Date': date,
                'Day_of_Week': day_of_week,
                'Crime_Occurrence': crime_occurred,
                'Crime_Severity': crime_severity
            })

        route_id += 1

    return pd.DataFrame(data)

# Generate the data
crime_data = generate_crime_data()

# Convert 'Day_of_Week' to numerical values (e.g., Monday = 0, Tuesday = 1)
day_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
crime_data['Day_of_Week'] = crime_data['Day_of_Week'].map(day_map)

# Prepare features and target variable
X = crime_data[['Route_ID', 'Day_of_Week']]
y = crime_data['Crime_Occurrence']

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Check accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

def build_graph(crime_data):
    G = nx.Graph()

    # Add edges with weights (average crime severity as weight)
    for _, row in crime_data.iterrows():
        if row['Crime_Occurrence'] == 1:
            G.add_edge(row['Origin'], row['Destination'], weight=row['Crime_Severity'])

    return G

def update_graph_weights(G, clf, day_of_week):
    day_num = day_map[day_of_week]

    for edge in G.edges:
        route_id = crime_data[(crime_data['Origin'] == edge[0]) & (crime_data['Destination'] == edge[1])]['Route_ID'].unique()
        if route_id.size > 0:  # Check if route_id has any elements
            X_new = np.array([[route_id[0], day_num]])
            crime_pred = clf.predict(X_new)[0]
            if crime_pred == 0:
                G[edge[0]][edge[1]]['weight'] = 0
            else:
                G[edge[0]][edge[1]]['weight'] = crime_data[(crime_data['Origin'] == edge[0]) &
                                                             (crime_data['Destination'] == edge[1])]['Crime_Severity'].mean()

def find_safest_route(G, origin, destination):
    try:
        path = nx.dijkstra_path(G, source=origin, target=destination, weight='weight')
        return path
    except nx.NetworkXNoPath:
        return "No available route between these towns."

# Build and update graph
G = build_graph(crime_data)

# Example: Update graph and find the safest route from Nairobi to Mombasa on a Tuesday
update_graph_weights(G, clf, 'Tuesday')
safest_route = find_safest_route(G, 'Nairobi', 'Mombasa')
print(f"The safest route from Nairobi to Mombasa on Tuesday is: {safest_route}")

# Visualization

def draw_graph(G, path=None):
    pos = nx.spring_layout(G)  # Position nodes using the Fruchterman-Reingold force-directed algorithm

    plt.figure(figsize=(12, 8))

    # Draw nodes and edges
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

    if path:
        # Highlight the safest path
        path_edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='r', width=2.0)

    nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif')

    plt.title('Graph Visualization with Safest Route and Edge Weights')
    plt.show()

# Draw the graph and highlight the safest route with edge weights
draw_graph(G, safest_route)


In [5]:
import pandas as pd
import numpy as np
import folium
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

# Define the towns and their coordinates
towns = {
    'Nairobi': (-1.286389, 36.817223),
    'Mombasa': (-4.043477, 39.668206),
    'Nakuru': (-0.303099, 36.066285),
    'Eldoret': (0.514277, 35.269065),
    'Kisumu': (-0.091702, 34.767956),
    'Nyeri': (-0.421250, 36.945752),
    'Machakos': (-1.516820, 37.266485),
    'Kericho': (-0.366690, 35.291340),
    'Meru': (-0.047200, 37.648000),
    'Kitale': (1.002559, 34.986032),
    'Garissa': (-0.456550, 39.664640),
    'Isiolo': (0.353073, 37.582666),
    'Bungoma': (0.591278, 34.564658),
    'Wajir': (1.737327, 40.065940),
    'Mandera': (3.930530, 41.855910),
    'Malindi': (-3.219186, 40.116944),
    'Lamu': (-2.271189, 40.902012),
    'Thika': (-1.033349, 37.069328),
    'Namanga': (-2.545290, 36.792530),
    'Kitui': (-1.374818, 38.010555),
    'Naivasha': (-0.707222, 36.431944),
    'Narok': (-1.078850, 35.860000),
    'Busia': (0.4605, 34.1115),
    'Bomet': (-0.7827, 35.3428),
    'Marsabit': (2.3342, 37.9891),
    'Voi': (-3.3962, 38.5565)
}

# Define multiple routes with at least 3 paths leading to each town
routes = [
    ('Nairobi', 'Mombasa', 'A109'),
    ('Nairobi', 'Nakuru', 'A104'),
    ('Nairobi', 'Eldoret', 'A104'),
    ('Nairobi', 'Nyeri', 'B5'),
    ('Nairobi', 'Garissa', 'A3'),
    ('Mombasa', 'Malindi', 'A14'),
    ('Mombasa', 'Garissa', 'B8'),
    ('Nakuru', 'Eldoret', 'A104'),
    ('Nakuru', 'Kericho', 'B1'),
    ('Nakuru', 'Nyeri', 'B5'),
    ('Eldoret', 'Kitale', 'B2'),
    ('Eldoret', 'Kisumu', 'A104'),
    ('Kisumu', 'Bungoma', 'A1'),
    ('Kisumu', 'Busia', 'A1'),
    ('Kisumu', 'Kericho', 'B1'),
    ('Nyeri', 'Nairobi', 'B5'),
    ('Nyeri', 'Meru', 'B6'),
    ('Garissa', 'Wajir', 'A3'),
    ('Garissa', 'Mandera', 'A3'),
    ('Kitale', 'Bungoma', 'A1'),
    ('Machakos', 'Nairobi', 'B6'),
    ('Machakos', 'Mombasa', 'A109'),
    ('Machakos', 'Kitui', 'B7'),
    ('Meru', 'Isiolo', 'A2'),
    ('Meru', 'Nairobi', 'B6'),
    ('Kericho', 'Nakuru', 'B1'),
    ('Kericho', 'Kisumu', 'B1'),
    ('Isiolo', 'Marsabit', 'A2'),
    ('Isiolo', 'Garissa', 'A2'),
    ('Bungoma', 'Kitale', 'A1'),
    ('Bungoma', 'Busia', 'A1'),
    ('Wajir', 'Garissa', 'A3'),
    ('Wajir', 'Mandera', 'A3'),
    ('Mandera', 'Wajir', 'A3'),
    ('Thika', 'Nairobi', 'A2'),
    ('Thika', 'Nyeri', 'A2'),
    ('Naivasha', 'Nairobi', 'B3'),
    ('Naivasha', 'Narok', 'C88'),
    ('Lamu', 'Malindi', 'C112')
]

# Generate simulated crime data
def generate_crime_data():
    data = []
    for origin, destination, road_name in routes:
        origin_lat, origin_lon = towns[origin]
        dest_lat, dest_lon = towns[destination]

        for day in range(730):  # 2 years of data
            day_of_week = day % 7  # Day of the week (0=Monday, 6=Sunday)
            hour_of_day = random.randint(0, 23)  # Random hour of the day
            crime_occurred = np.random.choice([0, 1], p=[0.85, 0.15])  # 15% chance of crime

            data.append({
                'Origin': origin,
                'Destination': destination,
                'Road_Name': road_name,
                'Day_of_Week': day_of_week,
                'Hour_of_Day': hour_of_day,
                'Crime_Occurred': crime_occurred,
                'Origin_Lat': origin_lat,
                'Origin_Lon': origin_lon,
                'Destination_Lat': dest_lat,
                'Destination_Lon': dest_lon
            })
    return pd.DataFrame(data)

crime_data = generate_crime_data()

# Train a crime prediction model
X = crime_data[['Day_of_Week', 'Hour_of_Day']]
y = crime_data['Crime_Occurred']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Function to find the safest route
def find_safest_route(origin, destination, day_of_week, hour_of_day):
    G = nx.Graph()

    for _, row in crime_data.iterrows():
        G.add_edge(row['Origin'], row['Destination'], weight=row['Crime_Occurred'])

    safest_path = nx.shortest_path(G, source=origin, target=destination, weight='weight')

    print(f"Safest Route from {origin} to {destination}: {safest_path}")

# Example usage:
find_safest_route('Nairobi', 'Kisumu', day_of_week=2, hour_of_day=14)


In [22]:
import pandas as pd
import numpy as np
import folium
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
import warnings
warnings.filterwarnings("ignore")

# Define the towns and their coordinates
towns = {
    'Nairobi': (-1.286389, 36.817223),
    'Mombasa': (-4.043477, 39.668206),
    'Nakuru': (-0.303099, 36.066285),
    'Eldoret': (0.514277, 35.269065),
    'Kisumu': (-0.091702, 34.767956),
    'Nyeri': (-0.421250, 36.945752),
    'Machakos': (-1.516820, 37.266485),
    'Kericho': (-0.366690, 35.291340),
    'Meru': (-0.047200, 37.648000),
    'Kitale': (1.002559, 34.986032),
    'Garissa': (-0.456550, 39.664640),
    'Isiolo': (0.353073, 37.582666),
    'Bungoma': (0.591278, 34.564658),
    'Wajir': (1.737327, 40.065940),
    'Mandera': (3.930530, 41.855910),
    'Malindi': (-3.219186, 40.116944),
    'Lamu': (-2.271189, 40.902012),
    'Thika': (-1.033349, 37.069328),
    'Namanga': (-2.545290, 36.792530),
    'Kitui': (-1.374818, 38.010555),
    'Naivasha': (-0.707222, 36.431944),
    'Narok': (-1.078850, 35.860000),
    'Busia': (0.4605, 34.1115),
    'Bomet': (-0.7827, 35.3428),
    'Marsabit': (2.3342, 37.9891),
    'Voi': (-3.3962, 38.5565)
}

# Define multiple routes with at least 3 paths leading to each town
routes = [
    ('Nairobi', 'Mombasa', 'A109'),
    ('Nairobi', 'Nakuru', 'A104'),
    ('Nairobi', 'Eldoret', 'A104'),
    ('Nairobi', 'Nyeri', 'B5'),
    ('Nairobi', 'Garissa', 'A3'),
    ('Mombasa', 'Malindi', 'A14'),
    ('Mombasa', 'Garissa', 'B8'),
    ('Nakuru', 'Eldoret', 'A104'),
    ('Nakuru', 'Kericho', 'B1'),
    ('Nakuru', 'Nyeri', 'B5'),
    ('Eldoret', 'Kitale', 'B2'),
    ('Eldoret', 'Kisumu', 'A104'),
    ('Kisumu', 'Bungoma', 'A1'),
    ('Kisumu', 'Busia', 'A1'),
    ('Kisumu', 'Kericho', 'B1'),
    ('Nyeri', 'Nairobi', 'B5'),
    ('Nyeri', 'Meru', 'B6'),
    ('Garissa', 'Wajir', 'A3'),
    ('Garissa', 'Mandera', 'A3'),
    ('Kitale', 'Bungoma', 'A1'),
    ('Machakos', 'Nairobi', 'B6'),
    ('Machakos', 'Mombasa', 'A109'),
    ('Machakos', 'Kitui', 'B7'),
    ('Meru', 'Isiolo', 'A2'),
    ('Meru', 'Nairobi', 'B6'),
    ('Kericho', 'Nakuru', 'B1'),
    ('Kericho', 'Kisumu', 'B1'),
    ('Isiolo', 'Marsabit', 'A2'),
    ('Isiolo', 'Garissa', 'A2'),
    ('Bungoma', 'Kitale', 'A1'),
    ('Bungoma', 'Busia', 'A1'),
    ('Wajir', 'Garissa', 'A3'),
    ('Wajir', 'Mandera', 'A3'),
    ('Mandera', 'Wajir', 'A3'),
    ('Thika', 'Nairobi', 'A2'),
    ('Thika', 'Nyeri', 'A2'),
    ('Naivasha', 'Nairobi', 'B3'),
    ('Naivasha', 'Narok', 'C88'),
    ('Lamu', 'Malindi', 'C112')
]

# Generate simulated crime data
def generate_crime_data():
    data = []
    for origin, destination, road_name in routes:
        origin_lat, origin_lon = towns[origin]
        dest_lat, dest_lon = towns[destination]

        for day in range(730):  # 2 years of data
            day_of_week = day % 7  # Day of the week (0=Monday, 6=Sunday)
            hour_of_day = random.randint(0, 23)  # Random hour of the day
            crime_occurred = np.random.choice([0, 1], p=[0.85, 0.15])  # 15% chance of crime

            data.append({
                'Origin': origin,
                'Destination': destination,
                'Road_Name': road_name,
                'Day_of_Week': day_of_week,
                'Hour_of_Day': hour_of_day,
                'Crime_Occurred': crime_occurred,
                'Origin_Lat': origin_lat,
                'Origin_Lon': origin_lon,
                'Destination_Lat': dest_lat,
                'Destination_Lon': dest_lon
            })
    return pd.DataFrame(data)

crime_data = generate_crime_data()

# Train a crime prediction model
X = crime_data[['Day_of_Week', 'Hour_of_Day']]
y = crime_data['Crime_Occurred']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Function to predict crime risk for a specific road
def predict_crime_probability(road, day_of_week, hour_of_day):
    X_new = np.array([[day_of_week, hour_of_day]])
    predicted_risk = clf.predict_proba(X_new)[0][1]  # Probability of crime occurring
    return predicted_risk

# Find the safest route using predicted crime probabilities
def find_safest_route(origin, destination, day_of_week, hour_of_day):
    G = nx.Graph()

    # Add edges with predicted crime probabilities as weights
    for route in routes:
        o, d, road_name = route
        crime_prob = predict_crime_probability(road_name, day_of_week, hour_of_day)
        G.add_edge(o, d, weight=crime_prob, road=road_name)

    # Find the safest path (path with the lowest crime risk)
    safest_path = nx.shortest_path(G, source=origin, target=destination, weight='weight')

    # Build route description
    route_description = []
    for i in range(len(safest_path) - 1):
        edge_data = G.get_edge_data(safest_path[i], safest_path[i+1])
        route_description.append(f"Take {edge_data['road']} from {safest_path[i]} to {safest_path[i+1]}")

    # Output the safest route and its description
    print(f"Safest Route from {origin} to {destination}:")
    for step in route_description:
        print(step)

# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
find_safest_route('Nairobi', 'Mombasa', day_of_week=2, hour_of_day=14)


In [8]:
# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
find_safest_route('Nairobi', 'Busia', day_of_week=2, hour_of_day=14)

In [23]:
import folium

# Function to display the safest route on a map
def display_route_on_map(route):
    # Initialize the map, centered on Kenya
    map_center = [0.0236, 37.9062]  # Center of Kenya
    kenya_map = folium.Map(location=map_center, zoom_start=6)

    # Add markers and lines for the safest route
    for i in range(len(route) - 1):
        origin = route[i]
        destination = route[i + 1]

        # Get the coordinates of origin and destination
        origin_coords = towns[origin]
        destination_coords = towns[destination]

        # Add markers for the towns
        folium.Marker(location=origin_coords, popup=origin, icon=folium.Icon(color='blue')).add_to(kenya_map)
        folium.Marker(location=destination_coords, popup=destination, icon=folium.Icon(color='blue')).add_to(kenya_map)

        # Draw lines between the towns
        folium.PolyLine(locations=[origin_coords, destination_coords], color='green', weight=5).add_to(kenya_map)

    return kenya_map

# Modified function to find the safest route and display it on the map
def find_safest_route_and_display(origin, destination, day_of_week, hour_of_day):
    G = nx.Graph()

    # Add edges with predicted crime probabilities as weights
    for route in routes:
        o, d, road_name = route
        crime_prob = predict_crime_probability(road_name, day_of_week, hour_of_day)
        G.add_edge(o, d, weight=crime_prob, road=road_name)

    # Find the safest path (path with the lowest crime risk)
    safest_path = nx.shortest_path(G, source=origin, target=destination, weight='weight')

    # Build route description
    route_description = []
    for i in range(len(safest_path) - 1):
        edge_data = G.get_edge_data(safest_path[i], safest_path[i+1])
        route_description.append(f"Take {edge_data['road']} from {safest_path[i]} to {safest_path[i+1]}")

    # Print the safest route and its description
    print(f"Safest Route from {origin} to {destination}:")
    for step in route_description:
        print(step)

    # Display the route on the map
    route_map = display_route_on_map(safest_path)
    return route_map

# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
route_map = find_safest_route_and_display('Nairobi', 'Nyeri', day_of_week=2, hour_of_day=14)
route_map


In [13]:
# Example usage: Safest route from Nairobi to Mombasa on a Tuesday at 2 PM
route_map = find_safest_route_and_display('Nairobi', 'Kisumu', day_of_week=2, hour_of_day=23)
route_map

In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Define towns and their coordinates
towns = {
    "Nairobi": (-1.286389, 36.817223),
    "Mombasa": (-4.043477, 39.668206),
    "Kisumu": (-0.091702, 34.768001),
    "Nakuru": (-0.303099, 36.080026),
    "Eldoret": (0.520361, 35.269861),
    "Kericho": (-0.366260, 35.332378),
    "Malindi": (-3.217648, 40.116350),
    "Embu": (-0.540512, 37.454620),
    "Meru": (-0.048318, 37.641077),
    "Machakos": (-1.522042, 37.261926),
    "Kitale": (1.003473, 34.989696),
    "Busia": (0.455541, 34.055416),
    "Bungoma": (0.591063, 34.564691),
    "Garissa": (-0.455931, 39.658767),
    "Wajir": (1.747735, 40.070517),
    "Isiolo": (0.358584, 37.584078),
    "Lamu": (-2.270558, 40.900960),
    "Kitui": (-1.373310, 38.053635),
    "Marsabit":  (2.337138, 37.978206),
    "Thika": (-1.033357, 37.073919),
    "Kajiado": (-2.066394, 36.818769),
    "Moyale": (3.537635, 39.079045),
}

# Define routes
routes = [
    ("Nairobi", "Mombasa", "A109"),
    ("Nairobi", "Nakuru", "A104"),
    ("Nairobi", "Eldoret", "A104"),
    ("Nairobi", "Kisumu", "A1"),
    ("Nairobi", "Malindi", "A14"),
    ("Nairobi", "Machakos", "B6"),
    ("Nairobi", "Embu", "B6"),
    ("Mombasa", "Kisumu", "A1"),
    ("Mombasa", "Nakuru", "A2"),
    ("Kisumu", "Eldoret", "B1"),
    ("Kisumu", "Busia", "B1"),
    ("Eldoret", "Kitale", "B2"),
    ("Nakuru", "Kericho", "B1"),
    ("Nakuru", "Bungoma", "C88"),
    ("Nairobi", "Meru", "B6"),
    ("Nairobi", "Embu", "B7"),
    ("Machakos", "Kitui", "B7"),
    ("Garissa", "Wajir", "A3"),
    ("Isiolo", "Moyale", "A2"),
    ("Mombasa", "Lamu", "B8"),
    ("Thika", "Nakuru", "B4"),
    ("Kajiado", "Nairobi", "C61"),
    ("Kitale", "Kisumu", "B2"),
    ("Kericho", "Bungoma", "B4"),
]

# Generate synthetic crime data
def generate_crime_data():
    data = {
        'Origin': [],
        'Destination': [],
        'Route': [],
        'Day_of_Week': [],
        'Hour_of_Day': [],
        'Crime_Occurred': []
    }

    for origin, destination, road_name in routes:
        origin_lat, origin_lon = towns[origin]
        dest_lat, dest_lon = towns[destination]
        for day in range(7):  # Simulate for each day of the week
            for hour in range(24):  # Simulate each hour of the day
                crime_occurred = np.random.choice([0, 1], p=[0.8, 0.2])
                data['Origin'].append(origin)
                data['Destination'].append(destination)
                data['Route'].append(road_name)
                data['Day_of_Week'].append(day)
                data['Hour_of_Day'].append(hour)
                data['Crime_Occurred'].append(crime_occurred)

    return pd.DataFrame(data)

crime_data = generate_crime_data()

# Prepare features and labels
X = crime_data[['Route', 'Day_of_Week', 'Hour_of_Day']]
y = crime_data['Crime_Occurred']

# Encode categorical variables
X = pd.get_dummies(X, columns=['Route'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Save the trained model
joblib.dump(clf, 'crime_model.pkl')


In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib

# Define towns and routes
towns = {
    'Nairobi': (1.286389, 36.817223),
    'Mombasa': (-4.043477, 39.668206),
    'Kisumu': (-0.091702, 34.767956),
    'Eldoret': (0.515310, 35.269385),
    'Nakuru': (-0.303099, 36.065945),
    'Kericho': (-0.368236, 35.295677),
    'Kakamega': (0.340988, 34.754274),
    'Meru': (0.050017, 37.641069),
    'Embu': (-0.541609, 37.456048),
    'Kitale': (1.003835, 34.987351),
    'Malindi': (-3.163533, 40.116198),
    'Garissa': (-0.455636, 39.664804),
    'Machakos': (-1.516313, 37.259322),
    'Isiolo': (0.350498, 37.580693),
    'Moyale': (3.504122, 39.083116)
}

routes = [
    ('Nairobi', 'Mombasa', 'A109'),
    ('Nairobi', 'Kisumu', 'A104/A1'),
    ('Nairobi', 'Eldoret', 'A104'),
    ('Nairobi', 'Nakuru', 'A104'),
    ('Nairobi', 'Kericho', 'B1'),
    ('Nairobi', 'Kakamega', 'A104'),
    ('Nairobi', 'Meru', 'B6'),
    ('Nairobi', 'Embu', 'B7'),
    ('Nairobi', 'Kitale', 'A104'),
    ('Nairobi', 'Malindi', 'A109'),
    ('Nairobi', 'Garissa', 'A3'),
    ('Nairobi', 'Machakos', 'B6'),
    ('Nairobi', 'Isiolo', 'A2'),
    ('Nairobi', 'Moyale', 'A2'),
    ('Mombasa', 'Kisumu', 'A109'),
    ('Mombasa', 'Eldoret', 'A109'),
    ('Mombasa', 'Nakuru', 'A109'),
    ('Mombasa', 'Kericho', 'B8'),
    ('Mombasa', 'Kakamega', 'B8'),
    ('Mombasa', 'Meru', 'B8'),
    ('Mombasa', 'Embu', 'B8'),
    ('Mombasa', 'Kitale', 'B8'),
    ('Mombasa', 'Malindi', 'B8'),
    ('Mombasa', 'Garissa', 'B8'),
    ('Mombasa', 'Machakos', 'B8'),
    ('Mombasa', 'Isiolo', 'B8'),
    ('Mombasa', 'Moyale', 'B8'),
    ('Kisumu', 'Eldoret', 'A1'),
    ('Kisumu', 'Nakuru', 'A1'),
    ('Kisumu', 'Kericho', 'A1'),
    ('Kisumu', 'Kakamega', 'A1'),
    ('Kisumu', 'Meru', 'A1'),
    ('Kisumu', 'Embu', 'A1'),
    ('Kisumu', 'Kitale', 'A1'),
    ('Kisumu', 'Malindi', 'A1'),
    ('Kisumu', 'Garissa', 'A1'),
    ('Kisumu', 'Machakos', 'A1'),
    ('Kisumu', 'Isiolo', 'A1'),
    ('Kisumu', 'Moyale', 'A1'),
    ('Eldoret', 'Nakuru', 'A104'),
    ('Eldoret', 'Kericho', 'B1'),
    ('Eldoret', 'Kakamega', 'B1'),
    ('Eldoret', 'Meru', 'B1'),
    ('Eldoret', 'Embu', 'B1'),
    ('Eldoret', 'Kitale', 'B1'),
    ('Eldoret', 'Malindi', 'B1'),
    ('Eldoret', 'Garissa', 'B1'),
    ('Eldoret', 'Machakos', 'B1'),
    ('Eldoret', 'Isiolo', 'B1'),
    ('Eldoret', 'Moyale', 'B1'),
    ('Nakuru', 'Kericho', 'B1'),
    ('Nakuru', 'Kakamega', 'B1'),
    ('Nakuru', 'Meru', 'B1'),
    ('Nakuru', 'Embu', 'B1'),
    ('Nakuru', 'Kitale', 'B1'),
    ('Nakuru', 'Malindi', 'B1'),
    ('Nakuru', 'Garissa', 'B1'),
    ('Nakuru', 'Machakos', 'B1'),
    ('Nakuru', 'Isiolo', 'B1'),
    ('Nakuru', 'Moyale', 'B1'),
    ('Kericho', 'Kakamega', 'B1'),
    ('Kericho', 'Meru', 'B1'),
    ('Kericho', 'Embu', 'B1'),
    ('Kericho', 'Kitale', 'B1'),
    ('Kericho', 'Malindi', 'B1'),
    ('Kericho', 'Garissa', 'B1'),
    ('Kericho', 'Machakos', 'B1'),
    ('Kericho', 'Isiolo', 'B1'),
    ('Kericho', 'Moyale', 'B1'),
    ('Kakamega', 'Meru', 'B1'),
    ('Kakamega', 'Embu', 'B1'),
    ('Kakamega', 'Kitale', 'B1'),
    ('Kakamega', 'Malindi', 'B1'),
    ('Kakamega', 'Garissa', 'B1'),
    ('Kakamega', 'Machakos', 'B1'),
    ('Kakamega', 'Isiolo', 'B1'),
    ('Kakamega', 'Moyale', 'B1'),
    ('Meru', 'Embu', 'B1'),
    ('Meru', 'Kitale', 'B1'),
    ('Meru', 'Malindi', 'B1'),
    ('Meru', 'Garissa', 'B1'),
    ('Meru', 'Machakos', 'B1'),
    ('Meru', 'Isiolo', 'B1'),
    ('Meru', 'Moyale', 'B1'),
    ('Embu', 'Kitale', 'B1'),
    ('Embu', 'Malindi', 'B1'),
    ('Embu', 'Garissa', 'B1'),
    ('Embu', 'Machakos', 'B1'),
    ('Embu', 'Isiolo', 'B1'),
    ('Embu', 'Moyale', 'B1'),
    ('Kitale', 'Malindi', 'B1'),
    ('Kitale', 'Garissa', 'B1'),
    ('Kitale', 'Machakos', 'B1'),
    ('Kitale', 'Isiolo', 'B1'),
    ('Kitale', 'Moyale', 'B1'),
    ('Malindi', 'Garissa', 'B1'),
    ('Malindi', 'Machakos', 'B1'),
    ('Malindi', 'Isiolo', 'B1'),
    ('Malindi', 'Moyale', 'B1'),
    ('Garissa', 'Machakos', 'B1'),
    ('Garissa', 'Isiolo', 'B1'),
    ('Garissa', 'Moyale', 'B1'),
    ('Machakos', 'Isiolo', 'B1'),
    ('Machakos', 'Moyale', 'B1'),
    ('Isiolo', 'Moyale', 'B1')
]

# Generate historical crime data
def generate_crime_data():
    data = []
    for origin, destination, road_name in routes:
        origin_lat, origin_lon = towns[origin]
        dest_lat, dest_lon = towns[destination]
        for day in range(7):  # Simulate each day of the week
            for hour in range(24):  # Simulate each hour of the day
                crime_occurred = np.random.choice([0, 1], p=[0.8, 0.2])
                data.append({
                    'Origin': origin,
                    'Destination': destination,
                    'Road_Name': road_name,
                    'Day_of_Week': day,
                    'Hour_of_Day': hour,
                    'Crime_Occurred': crime_occurred
                })
    return pd.DataFrame(data)

crime_data = generate_crime_data()

# Prepare features and labels
X = crime_data[['Day_of_Week', 'Hour_of_Day']]
y = crime_data['Crime_Occurred']

# Train a RandomForest model
model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)

# Save the model
joblib.dump(model, 'crime_prediction_model.pkl')
